# ACESSANDO DADOS DO GOOGLE EARTH ENGINE - GEE
1. Obtenção dos dados (utilizando GEE):
  - Selecionar área de estudo;
  - Obtendo dados multiespectrais;
  - Exportando dados para o driver.

## Conectando Google Drive

In [ ]:
# Conectando Google Drive
from google.colab import drive
drive.mount('/content/drive')

## 1   Obtendo dados do Google Earth Engine (GEE).

Obtendo as imagens/rasters:
  - Dados Landsat
  
---

## 2.Importação de bibliotecas e Autenticação no Earth Engine

In [ ]:
# Importar as bibliotecas
import geemap
import numpy as np
import ee
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
# Autenticação padrão da conta do Google Earth Engine

ee.Authenticate()
ee.Initialize(project='ee-izadorasc')  ## 'project' é individual para cada usuário

## 3.Área de Estudo

In [ ]:
area = "projects/ee-izadorasc/assets/MAPBIOMAS/bacia_lajeado"
roi = ee.FeatureCollection(area).geometry()

center = roi.centroid().coordinates().getInfo()[::-1]  # [lat, lon]
m = geemap.Map(center=center, zoom=7)
m.addLayer(roi, {'color':'black'}, 'ROI - bacia_lajeado')
m

## 4.**Informações sobre os sensores**

**Reflectância de Superficíe:**

Landsat 9 OLI (Out 31, 2021 - present)

  USGS Landsat 9 Level 2, Collection 2, Tier 1
  
  sensor = "LANDSAT/LC09/C02/T1_L2"

scale factor = 2.75e-05
scale factor surface temperature = 0.00341802

**Reflectância no Topo da Atmosfera:**

Landsat 9 OLI (Out 31, 2021 - present)

  USGS Landsat 9 Collection 2 Tier 1 TOA Reflectance
  sensor = "LANDSAT/LC09/C02/T1_TOA"

In [ ]:
#Visualização dos dados

# Visualização composição de bandas - Landsat

paletteVisLS9_SR = {
  'min': 0,
  'max': 0.4,
  'gamma': 1,
  'opacity': 1,
  'bands': ['SR_B6', 'SR_B5', 'SR_B4']}

paletteVisLS9_TOA = {
  'min': 0,
  'max': 0.4,
  'gamma': 1,
  'opacity': 1,
  'bands': ['B6', 'B5', 'B4']}


In [ ]:
## Landsat 4, 5, 7, 8 and 9 for SR data
def maskLSclouds(image):

  # Os bits 3 e 5 são a sombra da nuvem e a nuvem, respetivamente, para os produtos Landsat 4-5-7-8 e 9 SR
  # Aqui definimos os valores dos bits em valores de bytes.

# Criando uma função para mascarar pixéis de nuvens e sombras de nuvens
# 1 corresponde a céu limpo
# 3 corresponde a sombra de nuvem
# 5 corresponde a nuvem

  #cloudShadowBitMask = ee.Number(1).pow(3).int()
  #cloudsBitMask = ee.Number(1).pow(5).int()
  qaMask = image.select('QA_PIXEL').bitwise_and(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Obter a banda de pixel QA da coleção.
  #qa = image.select('QA_PIXEL')

  ###Fator de escala
  #opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

  ###Fator de escala
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

  ##Produto resultante
  return image.addBands(opticalBands, None, True)\
  .addBands(thermalBands, None, True).updateMask(qaMask)\
  .updateMask(saturationMask)


#### **Imagens Landsat SR**

In [ ]:
# Definindo a coleção do LANDSAT

sensor = "LANDSAT/LC09/C02/T1_L2"

# Data
date_range = ['2024-01-01', '2024-12-31']


# Carregando dados
sr_collection = ee.ImageCollection(sensor) \
    .filterBounds(roi)\
    .filter(ee.Filter.lt('CLOUD_COVER', 1))\
    .filterDate(date_range[0], date_range[1])

# Define wet and dry season subsets


# Map the maskLSclouds function over each seasonal dataset and take the median of clear pixels.
img_comp = sr_collection.map(maskLSclouds).median()
# img_comp = sr_collection.median()

# Clip the composite images to the Region of Interest
clip_img_comp = img_comp.clip(roi)


# Carregando propriedades da visualização
Map = geemap.Map(center=center, zoom=10)

Map.addLayer(clip_img_comp, paletteVisLS9_SR, 'LS9-SR')

# Visualizando
Map.addLayerControl()
Map


In [ ]:
#Exportando imagem para o driver

task = ee.batch.Export.image.toDrive(
    image=clip_img_comp.select('SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'),
    description='Img_Landsat9_bcLajeado_2024', #nome do arquivo de imagem que será salvo
    folder='GEE_Exports', #nome da pasta que o raster (imagem) será salva no driver
    fileNamePrefix='Img_Landsat9_bcLajeado_2024',
    region=roi, #limite da área de estudo
    fileFormat= "GeoTIFF",
    crs= "EPSG:4326", #WGS84 - World Geodetic System 1984
    scale=30, #escala espacial da imagem em metros
    maxPixels=1e13
)

task.start()
print("Exportação imagem Landsat iniciada (Para a Pasta no Drive -> GEE_Exports).")

#### **Imagens Landsat TOA**

In [ ]:
# Definindo a coleção do LANDSAT

sensor = "LANDSAT/LC09/C02/T1_TOA"

# Data
date_range = ['2024-01-01', '2024-12-31']


# Carregando dados
sr_collection = ee.ImageCollection(sensor) \
    .filterBounds(roi)\
    .filter(ee.Filter.lt('CLOUD_COVER', 1))\
    .filterDate(date_range[0], date_range[1])

# Define wet and dry season subsets


# Map the maskLSclouds function over each seasonal dataset and take the median of clear pixels.
img_comp_toa = sr_collection.median()

# Clip the composite images to the Region of Interest
clip_img_comp_toa = img_comp_toa.clip(roi)


# Carregando propriedades da visualização
Map = geemap.Map(center=center, zoom=10)

Map.addLayer(clip_img_comp_toa, paletteVisLS9_TOA, 'LS9-TOA')

# Visualizando
Map.addLayerControl()
Map


In [ ]:
#Exportando imagem para o driver

task = ee.batch.Export.image.toDrive(
    image=clip_img_comp_toa.select('B2', 'B3', 'B4', 'B5', 'B6', 'B7'),
    description='Img_Landsat9_TOA_bcLajeado_2024', #nome do arquivo de imagem que será salvo
    folder='GEE_Exports', #nome da pasta que o raster (imagem) será salva no driver
    fileNamePrefix='Img_Landsat9_TOA_bcLajeado_2024',
    region=roi, #limite da área de estudo
    fileFormat= "GeoTIFF",
    crs= "EPSG:4326", #WGS84 - World Geodetic System 1984
    scale=30, #escala espacial da imagem em metros
    maxPixels=1e13
)

task.start()
print("Exportação imagem Landsat iniciada (Para a Pasta no Drive -> GEE_Exports).")

In [ ]:
# Mapa comparativo - SR e TOA

left_layer = geemap.ee_tile_layer(clip_img_comp, paletteVisLS9_SR, 'LS9-SR')
right_layer = geemap.ee_tile_layer(clip_img_comp_toa, paletteVisLS9_TOA, 'LS9-TOA')

Map = geemap.Map(center=center, zoom=10)

Map.split_map(left_layer, right_layer)
Map